In [14]:
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import matthews_corrcoef
import optuna
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Data Load

In [15]:
df = pd.read_hdf("data/ID18150/Day2/DataFrame_Imaging_dFF_18150_day2.h5")
# Get a list of columns whose names are of numerical type
numerical_columns = [col for col in df.columns if type(col) == int]

#Feature matrix
X = df[numerical_columns].values
t = df.Time.to_list()
y = df['Pump'].astype(int)
n_features = len(numerical_columns)

# Modelling

In [16]:
n_folds = 10
def objective(trial):
    kf = KFold(n_splits=n_folds)
    mcc_acc = []
    for split in kf.split(X):
        X_train, X_test = X[split[0]], X[split[1]]
        y_train, y_test = y[split[0]], y[split[1]]
        pca = PCA(n_components=trial.suggest_int('n_components', int(0.05*n_features), int(0.5*n_features)))
        pca.fit(X_train)
        X_train_pca = pca.transform(X_train)
        X_test_pca = pca.transform(X_test)
        clf = KNeighborsClassifier(n_neighbors=trial.suggest_int('n_neighbors', 1, 10))
        clf.fit(X_train_pca, y_train)
        y_pred = clf.predict(X_test_pca)
        mcc = matthews_corrcoef(y_test, y_pred)
        mcc_acc.append(mcc)
    return np.mean(mcc_acc)

In [17]:
study = optuna.create_study(direction='maximize', storage="sqlite:///data/ID18150/Day2/pca_classifier.db")
study.optimize(objective, n_trials=100)

[I 2024-11-06 20:02:58,630] A new study created in RDB with name: no-name-bb05a5a9-a9bd-471e-97fe-d1cfe7f7a278
[W 2024-11-06 20:03:00,971] Trial 0 failed with parameters: {'n_components': 175, 'n_neighbors': 10} because of the following error: ValueError('Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.').
Traceback (most recent call last):
  File "/home/icaro/Documents/doctorate/dimensionality-reduction/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_91579/3119129330.py", line 13, in objective
    clf.fit(X_train_pca, y_train)
  File "/home/icaro/Documents/doctorate/dimensionality-reduction/.venv/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.